# Análisis Exploratorio

En este cuaderno se analizará detalladamente la naturaleza de los datos del archivo **product.csv** proporcionado por la organización del Datathon

# Índice de Contenidos
1. **Importación de paquetes**
2. **Carga de datos**
3. **Visualización de datos**
4. **Conclusiones**

## Importación de paquete

El primer paso es importar los módulos que nos facilitarán el trabajo.

In [26]:
from IPython.display import Image
from urllib.error import HTTPError
import pandas as pd
import random
import re
import plotly.express as px

# Semilla para realizar experimentos reproducibles
seed = 125
random.seed(seed)

## Carga de datos

Abrimos los ficheros y los cargamos para empezar a manejarlos y extraer información relevante.

In [27]:
df_products = pd.read_csv("../Data/products.csv")
df_products.sample(5, random_state=seed)

,product_id,sku,name,marca_value,short_description,analytic_category,picture
23485,76086,MISUPI,SuperPack personalizado meu Pipo,Mi Pipo,<p>El SuperPack personalizado de Mi Pipo es id...,NaN,https://www.mifarma.es/media/catalog/product/m...
27586,74774,41401,2x2ml soro & camaleão de cor mágica,Camaleon,<p>Si tu piel necesita un efecto lifting inmed...,Cosmética y Belleza,https://www.mifarma.es/media/catalog/product/4...
2777,4388,152307,Aquilea Gases 60 Comprimidos,Aquilea,<p>Solución natural a los gases gracias a su c...,Herbolario,https://www.mifarma.es/media/catalog/product/1...
12655,79092,BN013,Colageno Marino + Silicio Organico Binature 18...,Binature,<p>Complemento alimenticio con Colágeno marino...,Herbolario,https://www.mifarma.es/media/catalog/product/b...
12783,61316,44701381,Tommee Tippee Easy Drink Taza Aprendizaje 230m...,Tommee tippee,<p>Taza con pajita fabricada en suave silicona...,Infantil,https://www.mifarma.es/media/catalog/product/4...


In [28]:
print(f"Dimensiones de los datos: nº columnas ({df_products.shape[0]}) y nº filas ({df_products.shape[1]})")

Dimensiones de los datos: nº columnas (27953) y nº filas (7)


El *dataset* se compone de **7** columnas/características y cerca de unos **30.000** registros, de los cuales podemos ver que algunos son nulos. Más adelante veremos que datos faltan y determinaremos la forma de salvaguardar estos casos.

In [4]:
df_products.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27953 entries, 0 to 27952
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   product_id         27953 non-null  int64 
 1   sku                27953 non-null  object
 2   name               27939 non-null  object
 3   marca_value        27921 non-null  object
 4   short_description  27930 non-null  object
 5   analytic_category  24416 non-null  object
 6   picture            27947 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.5+ MB


Vemos que el tipo de datos de algunas de las columnas puede definirse simplemente como *string* en lugar de *object*

In [5]:
df_products.describe(include=['object'])

,sku,name,marca_value,short_description,analytic_category,picture
count,27953,27939,27921,27930,24416,27947
unique,23216,27862,792,20597,10,23175
top,80890556,Avene Couvrance Compacto,Suavinex,"<p>Bebida instantánea sin azúcar, muy baja en ...",Cosmética y Belleza,https://www.mifarma.es/media/catalog/product/1...
freq,4,4,761,58,8336,4


De un vistazo vemos las características de las distintas variables:
- Hay nombres de productos que se repiten, al igual que sus descripciones. 
- Resulta *"extraño"* que el nº de descripciones no concuerde con el número de productos.
- Hay productos que al parecer no tienen fotos
- Hay casi 800 marcas de productos diferentes
- Llama la atención que la descripciones estén contenidas en etiquetas de párrafo HTML

In [6]:
percent_missing = df_products.isnull().sum() * 100 / df_products.shape[0]
df_missing_values = pd.DataFrame({ 
    'column_name': df_products.columns,
    'percent_missing': percent_missing
})
df_missing_values.sort_values(by='percent_missing',ascending=False,inplace=True)
print(df_missing_values.to_string(index=False))

      column_name  percent_missing
analytic_category        12.653382
      marca_value         0.114478
short_description         0.082281
             name         0.050084
          picture         0.021465
       product_id         0.000000
              sku         0.000000


Efectivamente tal y como sospechábamos desde un principio, dado que el número de registros totales no concuerda con el recuento para algunas de las variables, que existe cierto porcentaje de valores perdidos/nulos. Por lo general, no es muy elevado, a excepción de la variable **analytic_category**.

## Visualizaciones de los datos

En este apartado se detallará más en profundidad las caracaterísticas de cada una de las variables, así como también las posibles relaciones entre ellas.

### Nombres de productos

La primera pregunta que nos surge es saber cuales son los productos que más se repiten y cuántas veces, así que vamos a responderla con algunas operaciones básicas.

In [7]:
# Nombres de los productos repetidos y cuantas veces se repiten
df_repeated_prods = df_products['name'].value_counts().loc[lambda x: x > 1 ].reset_index()
df_repeated_prods.columns = ['name', 'count']
df_repeated_prods

,name,count
0,Avene Couvrance Compacto,4
1,Medela Embudo Personal Fit,3
2,IAP Pharma Perfume mulher n º 150 ml,3
3,Medela Top de Extracción Fácil Sujetador Manos...,3
4,Babé Stick Labial 4 Gramos,2
...,...,...
67,Farmatint 8N Rubio Claro 130ml,2
68,Chupete Silicona Little Cutie Canpol Babies 6-...,2
69,Mosquitox Pulsera Color Rosa + 3 Pastillas,2
70,Portachupetes Suavinex Duo Blanco,2


Vemos que hay **72** productos repetidos, algunos (muy pocos) incluso se repiten más de dos veces.

In [8]:
# Nos quedamos solamente con los nombres de los productos
names_rep_prods = df_repeated_prods['name'].to_list()
names_rep_prods[:10]

['Avene Couvrance Compacto',
 'Medela Embudo Personal Fit',
 'IAP Pharma Perfume mulher n º 150 ml',
 'Medela Top de Extracción Fácil Sujetador Manos Libres',
 'Babé Stick Labial 4 Gramos',
 'Gel Masaje Madagascar Sweetness Control 200ml',
 'Nivea Crema 150ml',
 'Gel Contorno de Ojos Sesderma Men 15ml',
 'Babaria Crema Facial Antiarrugas Rosa Mosqueta 50ml',
 'Sebamed Champú Dermatológico Anticaspa ']

Ahora bien, ¿por qué hay productos repetidos? 

In [30]:
df_products[df_products['name'].isin(names_rep_prods)].sort_values(by='product_id').sample(10)

,product_id,sku,name,marca_value,short_description,analytic_category,picture
13170,11068,154277,ARMOLIPID PLUS 20 Comprimidos,Meda,<p>Suplemento alimenticio que ayuda a reducir ...,Herbolario,https://www.mifarma.es/media/catalog/product/1...
26924,71104,CB23263X.niña,Chupete Silicona Little Cutie Canpol Babies 6-...,Canpol Babies,<p>Chupete con tetina anatómica de silicona in...,NaN,https://www.mifarma.es/media/catalog/product/c...
20801,84100,181140,Potito Frutas de Temporada con Galletas Nutrib...,Nutribén,<p>El Potito Frutas de Temporada con Galletas ...,Infantil,https://www.mifarma.es/media/catalog/product/1...
15782,4488,serv201939.232504-232504,Sebamed Aceite de Ducha,Sebamed,<p>Higiene diaria en pieles secas y sensibles....,NaN,https://www.mifarma.es/media/catalog/product/s...
21881,72480,159472x2,Babé Stick Labial 4 Gramos,Babe,<p>Stick labial de textura cremosa que hidrata...,Cosmética y Belleza,https://www.mifarma.es/media/catalog/product/1...
861,1745,236265,Germisdin Higiene Corporal Piel Seca 1 Litro,Isdin,<p><strong>Gel de higiene diaria sin jabón</st...,Higiene,https://www.mifarma.es/media/catalog/product/2...
6159,35799,213000-x3,Epaplus Magnesio + Acido Hialuronico + Vitamin...,Epa-plus,<p><strong>Complemento alimenticio</strong> el...,Herbolario,https://www.mifarma.es/media/catalog/product/2...
25588,37398,176510,IAP Pharma Perfume para mulher 30ml não.,iap pharma,<p>Fragancia de mujer con notas olfativas afru...,NaN,https://www.mifarma.es/media/catalog/product/1...
18759,75475,186977,Gel Masaje Mediterranean Sea Control 200ml,Control,<p>Control Gel Masaje es perfecto para momento...,Vida Íntima,https://www.mifarma.es/media/catalog/product/1...
23876,47527,70408,Solgar Vitamina C 500 mg 90 comp masticables,Solgar,<p>Vitamina C masticable con sabor frambuesa. ...,Nutrición,https://www.mifarma.es/media/catalog/product/7...


In [31]:
df_repeated_prods['count'].sum()

149

Es curioso que a pesar de tener el mismo nombre, marca, descripción y categoría, el código de los productos cambia para todos. Quizás es un de las razones por las que se repiten. 

**NOTA** 

Al haber muchos productos (~150) mostrar todas las combinaciones resulta engorroso. Jugando con diferentes tipos de *sorting* (filtrando por diferentes columnas) nos hemos percatado de otras cosas como:
- Los **product_id** no coinciden aun siendo todos los demás campos iguales
- Algunos campos son nulos, predominantemente la **analytic_category**
- Tienen diferente descripción

### Marcas

El catálogo de productos es muy amplio, al igual que las marcas, así que veamos como es su distribución entre los productos.

In [11]:
df_products["marca_value"].value_counts().head(20)

Suavinex          761
Chicco            698
Sesderma          474
Isdin             461
Nuk               405
Apivita           372
La Roche Posay    364
Mam               350
Soria Natural     348
InterApothek      322
Nuxe              311
Beter             302
Arkopharma        287
Otras Marcas      287
Canpol Babies     285
Uriage            267
Ziaja             263
Vichy             259
Th Pharma         252
Tommee tippee     251
Name: marca_value, dtype: int64

El **top 20** de marcas en el catálogo están relacionadas sobretodo con *productos de bebé* (Suavinex, Chicco, Nuk, Mam ...) y productos de cosmética (Sesderma, Isdin, La Roche Posay ...)

In [12]:
df_products["marca_value"].value_counts().tail(20)

Arkosol                  1
Elimax                   1
Farmavital               1
Sangali                  1
Keravit                  1
Damira                   1
Cryopharma               1
Dermaclin                1
Vitalcare                1
Be Cool                  1
Compotoirs Compagnies    1
MAF                      1
Sloan                    1
Suntribe                 1
Redusan                  1
Nevasona                 1
Simplisse                1
Inglesina                1
Depilduch                1
Khadi                    1
Name: marca_value, dtype: int64

En cambio, las marcas menos frecuentes están relacionadas con productos muy específicos o con poca variedad de producto dentro de su gama como pueden ser antipiojos (Elimax), vitaminas (Vitalcare), bronceadores (Arkosol), etc.

### Descripciones

Tal y como se ha comentado, resulta un tanto extraño que no haya el mismo número de descripciones que de productos. Así que veamos detalladamente a que se debe esto. Por otro lado, cabe mencionar algo que se nos ha pasado comentar antes y es el hecho de que parece que los las descripciones se han obtenido a través de alguna técnica de *web scraping*, ya que conservan la etiqueta **p** propia de un documento HTML

In [13]:
df_descriptions = df_products["short_description"].value_counts().reset_index()
df_descriptions.columns = ['short_description', 'count']
df_descriptions

,short_description,count
0,"<p>Bebida instantánea sin azúcar, muy baja en ...",58
1,<p>Biberón anticólico con un color muy atracti...,29
2,<p>Tinte permanente para el cabello sin PPD.\n...,28
3,<p>Diseño innovador y divertido\n</p>,25
4,<p>Farmatint es el gel de coloración permanene...,24
...,...,...
20592,<p>Aceite Seco Protección Media: Nutritivo y i...,1
20593,<p>Gel fresco calmante al Monoï. Hidratante y ...,1
20594,<p>Mordedor ligero fase 2\n</p>,1
20595,<p>Un nuevo mundo de sabores en la alimentació...,1


Lo primero de lo que uno se da cuenta es que una de las descripciones se repite bastantes veces. Veamos a qué producto pertenece y si ese producto es uno de los que más se repite dentro del *dataset*

In [14]:
# Nos quedamos solamente con las descripciones
descriptions = df_descriptions.loc[df_descriptions['count'] > 1]['short_description'].to_list()
descriptions[:10]

['<p>Bebida instantánea sin azúcar, muy baja en calorías y carbohidratos. Es un refresco saludable para toda la familia, sin conservantes artificiales y con aromas y saborizantes naturales. Con cada sobre, podrás hacer 1,5 litros de bebida.\n</p><p>- Con Stevia\n</p><p>- Sin gluten\n</p>',
 '<p>Biberón anticólico con un color muy atractivo y cuatro flujos diferentes, que convierte la hora de la comida en un apasionante juego\n</p>',
 '<p>Tinte permanente para el cabello sin PPD.\n</p>',
 '<p>Diseño innovador y divertido\n</p>',
 '<p>Farmatint es el gel de coloración permanenete que consigue un color natural, duradero y con una cobertura del 100% de las canas.\n</p>',
 '<p>Mini Pintauñas.\n</p>',
 '<p>Tinte para el cabello formulado sin amoniaco ni parabenos, especialmente indicado para la coloración del cuero cabelludo sensible.\n</p>',
 '<p>Fragancia de mujer con notas olfativas afrutadas\n</p>',
 '<p>Novedoso vaso de aprendizaje ideal para la transición del biberón a la autonomía de 

In [15]:
df_products.loc[df_products['short_description'] == descriptions[0]].sample(n=5,random_state=seed)

,product_id,sku,name,marca_value,short_description,analytic_category,picture
6961,65021,guarana27,Bebida Bolero Sabor Guarana con Stevia 1 Sobre,Bolero,"<p>Bebida instantánea sin azúcar, muy baja en ...",NaN,https://www.mifarma.es/media/catalog/product/g...
13925,65335,coco38,Bebida Bolero Sabor Coco con Stevia 1 Sobre,Bolero,"<p>Bebida instantánea sin azúcar, muy baja en ...",Nutrición,https://www.mifarma.es/media/catalog/product/c...
8895,65308,frutadagron47,Bebida Bolero Sabor Fruta de Dragon con Stevia...,Bolero,"<p>Bebida instantánea sin azúcar, muy baja en ...",Nutrición,https://www.mifarma.es/media/catalog/product/f...
12003,65060,piña14,Bebida Bolero Sabor Piña con Stevia 1 Sobre,Bolero,"<p>Bebida instantánea sin azúcar, muy baja en ...",Nutrición,https://www.mifarma.es/media/catalog/product/p...
2352,65039,sandia21,Bebida Bolero Sabor Sandia con Stevia 1 Sobre,Bolero,"<p>Bebida instantánea sin azúcar, muy baja en ...",Nutrición,https://www.mifarma.es/media/catalog/product/s...


Resulta que la descripción se repite porque se trata de la misma bebida, pero en realidad son productos diferentes dado que tienen sabores diferentes (intuimos por el nombre) y también diferents **product_id** y **sku**. Veamos otro ejemplo *aleatorio*(*).

In [16]:
random.seed(seed)
df_products.loc[df_products['short_description'] == descriptions[random.randint(0,len(descriptions)-1)]]

,product_id,sku,name,marca_value,short_description,analytic_category,picture
4295,8140,CAU141,Caudalie Espuma Limpiadora Fleur De Vigne Tama...,Caudalie,<p>Espuma limpiadora suave y formulada <strong...,Higiene,https://www.mifarma.es/media/catalog/product/c...
22579,8140,CAU141,Caudalie limpeza espuma viagens de tamanho Fle...,Caudalie,<p>Espuma limpiadora suave y formulada <strong...,Higiene,https://www.mifarma.es/media/catalog/product/c...


Este caso es aun más extraño, ya que lo único que cambia es el nombre, pero ni el **product_id** ni el **sku** se ven afectados. Suponemos que es una peculiardid a la hora de organizar el catálogo por parte de la propia empresa

### Categorías

Esta es la variable con mayor porcentaje de valores perdidos. Dado que el otro archivo **products_categories.csv** detalla la categoría de los productos a través de la clave **sku**, no debería ser un problema recuperar los valores que faltan.

In [17]:
print(f"Número de categorías : {df_products['analytic_category'].nunique()}")

Número de categorías : 10


Como ya vimos en la descripción general del *dataset*, tan solo existen **10** categorías entre las que se reparten el catálogo de productos.

In [18]:
df_categories = df_products["analytic_category"].value_counts().reset_index()
df_categories.columns = ['analytic_category', 'count']
df_categories

fig = px.pie(df_categories, values='count', names='analytic_category', title='Distribución de las categorías de productos',color_discrete_sequence=px.colors.qualitative.Set3)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

Por los productos más repetidos, podíamos ya intuir por donde "irian los tiros" en cuanto a las categorías. En primer lugar, *"Cosmética y Belleza"* forma el grupo más amplio de productos, siguiendole los de *"Higiene"* y los *"Infantiles"*. Recordemos que en el **top** de productos más repetidos encontramos sobre todo cosméticos y artículos para bebés.

In [19]:
# Nos quedamos solamente con las categorias
categories = df_categories['analytic_category'].to_list()
categories

['Cosmética y Belleza',
 'Higiene',
 'Infantil',
 'Herbolario',
 'Nutrición',
 'Ortopedia',
 'Vida Íntima',
 'Óptica',
 'Perfumeria',
 'Veterinaria']

In [20]:
df_pr_cat = df_products[df_products['analytic_category'] == categories[0]][['name','marca_value']]
print(f"Hay {df_pr_cat['name'].nunique()} productos dentro de {categories[0]}")
print(f"Hay {df_pr_cat['marca_value'].nunique()} marcas dentro de {categories[0]}")
print('------------------')
print("Los productos más comunes dentro de la categoría\n")
products = df_pr_cat['name'].value_counts()[:5]
print(products)
print('------------------')
print("Las marcas más comunes (nºprod) dentro de la categoría\n")
marcas = df_pr_cat['marca_value'].value_counts()[:5]
print(marcas)

Hay 8311 productos dentro de Cosmética y Belleza
Hay 343 marcas dentro de Cosmética y Belleza
------------------
Los productos más comunes dentro de la categoría

Phytolium 4 Anti-caida 12 Dosis (-20% DTO)                                          2
NeoStrata Pack Skin Active Matrix Support SPF30 50ml + Cellular Restoration 50ml    2
Naturia Champu Uso Frecuente RENE Furterer 500ml                                    2
Hidrotelial Bálsamo Reparador Nariz y Labios SPF15 10 ml                            2
Vichy Liftactiv Flexilift Maquillaje 30 ml Color Nº55 Bronce                        2
Name: name, dtype: int64
------------------
Las marcas más comunes (nºprod) dentro de la categoría

Sesderma          360
La Roche Posay    271
Isdin             270
Beter             244
Nuxe              202
Name: marca_value, dtype: int64


In [21]:
df_pr_cat = df_products[df_products['analytic_category'] == categories[1]][['name','marca_value']]
print(f"Hay {df_pr_cat['name'].nunique()} productos dentro de {categories[1]}")
print(f"Hay {df_pr_cat['marca_value'].nunique()} marcas dentro de {categories[1]}")
print('------------------')
print("Los productos más comunes dentro de la categoría\n")
products = df_pr_cat['name'].value_counts()[:5]
print(products)
print('------------------')
print("Las marcas más comunes (nºprod) dentro de la categorían\n")
marcas = df_pr_cat['marca_value'].value_counts()[:5]
print(marcas)

Hay 5390 productos dentro de Higiene
Hay 369 marcas dentro de Higiene
------------------
Los productos más comunes dentro de la categoría

Champu Cebolla Nuggela 250 ml                          2
Germisdin Higiene Corporal Piel Seca 1 Litro           2
Bioderma Atoderm Gel Moussant 500 ml                   2
Fluorkin Pasta Dentifrica Junior Cola 75ml             2
Babaria Crema Facial Antiarrugas Rosa Mosqueta 50ml    2
Name: name, dtype: int64
------------------
Las marcas más comunes (nºprod) dentro de la categorían

Lacer      174
Kin        133
Apivita    127
Isdin      123
Ziaja      120
Name: marca_value, dtype: int64


Es curioso que a pesar de que *Cosmética y Belleza* sea la categoría *"más común"*, alberga menos marcas que la segunda categoría más común que es *Higiene*.

### Imágenes 

La última variable que nos queda por explorar son los enlaces proporcionados de cada uno de los productos. Lo único que podemos hacer es ver como se compone la **URL** y ver si tenemos acceso a realizar peticiones a estas mismas.

In [22]:
print("Extensión de los archivos de imagen\n")
print("Hay {} de tipo JPG".format(df_products['picture'].str.count(r'(\.jpg)').sum()))
print("Hay {} de tipo PNG".format(df_products['picture'].str.count(r'(\.png)').sum()))
print("Hay {} de tipo BMP".format(df_products['picture'].str.count(r'(\.bmp)').sum()))
print("Hay {} de tipo SVG".format(df_products['picture'].str.count(r'(\\.svg)').sum()))

Extensión de los archivos de imagen

Hay 27945.0 de tipo JPG
Hay 0.0 de tipo PNG
Hay 0.0 de tipo BMP
Hay 0.0 de tipo SVG


In [23]:
# Nos quedamos con algunas imágenes aleatorias*
pictures = df_products['picture'].sample(n=10,random_state=seed).to_list()
pictures

['https://www.mifarma.es/media/catalog/product/m/i/misupi_.jpg',
 'https://www.mifarma.es/media/catalog/product/4/1/41401_.jpg',
 'https://www.mifarma.es/media/catalog/product/1/5/152307_.jpg',
 'https://www.mifarma.es/media/catalog/product/b/n/bn013_.jpg',
 'https://www.mifarma.es/media/catalog/product/4/4/44701381_.jpg',
 'https://www.mifarma.es/media/catalog/product/2/3/232330_.jpg',
 'https://www.mifarma.es/media/catalog/product/4/2/425851_.jpg',
 'https://www.mifarma.es/media/catalog/product/2/4/246253.duplo_.jpg',
 'https://www.mifarma.es/media/catalog/product/1/7/170203_.jpg',
 'https://www.mifarma.es/media/catalog/product/m/n/mn11010u_.jpg']

Las fotos están organizadas por carpetas con codificación alfanumérica en las hojas del árbol de directorios, partiendo que la raíz es **media** que deriva en **catalog** y este a su vez en **product**. Veamos si coincide con su **product_id** o quizás con **sku**.

In [24]:
random.seed(seed)
picture = pictures[random.randint(0,len(pictures)-1)] # Imagen al azar* 
pattern = re.compile(r'([\w]+)(\.jpg)') # Buscamos el nombre del fichero con la extenstion
file_name = pattern.search(picture).group(1) # Sacamos solamente el grupo del nombre
print(f"El nombre del fichero es {file_name}") 
print('----------------')
# Probamos con product_idp --> quitamos el "slash" nombre
print("Filtrando por product_id\n")
print(df_products[(df_products['product_id'] == file_name[:-1]) & (df_products['picture'] == picture)].to_records())
print('----------------')
# Probamos con sku  --> quitamos el "slash" nombre
print("Filtrando por sku\n")
print(df_products[(df_products['sku'] == file_name[:-1]) & (df_products['picture'] == picture)].to_records())

El nombre del fichero es bn013_
----------------
Filtrando por product_id

[]
----------------
Filtrando por sku

[]


Vemos efectivamente como el nombre del fichero coincide con el código del producto. 

**NOTA**

Al haber tantos productos, consideramos innecesario ir uno a uno comprobando si esta regla se cumple. Por tanto, daremos por hecho que no siempre se dará el caso.

In [25]:
for picture in pictures:
    try:
        Image(url=picture,embed=True)
    except HTTPError as e:
        print(e)
    print(picture)

HTTP Error 410: Gone
https://www.mifarma.es/media/catalog/product/m/i/misupi_.jpg
HTTP Error 410: Gone
https://www.mifarma.es/media/catalog/product/4/1/41401_.jpg
HTTP Error 410: Gone
https://www.mifarma.es/media/catalog/product/1/5/152307_.jpg
HTTP Error 410: Gone
https://www.mifarma.es/media/catalog/product/b/n/bn013_.jpg
HTTP Error 410: Gone
https://www.mifarma.es/media/catalog/product/4/4/44701381_.jpg
HTTP Error 410: Gone
https://www.mifarma.es/media/catalog/product/2/3/232330_.jpg
HTTP Error 410: Gone
https://www.mifarma.es/media/catalog/product/4/2/425851_.jpg
HTTP Error 410: Gone
https://www.mifarma.es/media/catalog/product/2/4/246253.duplo_.jpg
HTTP Error 410: Gone
https://www.mifarma.es/media/catalog/product/1/7/170203_.jpg
HTTP Error 410: Gone
https://www.mifarma.es/media/catalog/product/m/n/mn11010u_.jpg


Parece ser que no es posible recuperar ningunas de las imágenes debido a un error **HTTP 410** que nos indica que el fichero ya no está disponible. Consideramos que esto ocurrirá con todas las fotos, no solamente con el *sample*, ya que se tratan de datos de 2017 y 2018, por lo que ha habido tiempo suficiente como para que MiFarma haya cambiado tanto de productos, como de sus imágenes por algunas más actuales.

## Conclusiones 

De este análisis deducimos lo siguiente:

- Existe un **12%** de valores perdidos para el atributo *analytic_category*
- Las descripciones contienen etiquetas HTML
- Las URLs del campo de *picture* están obsoletos
- Hay una gran variedad de marcas de productos, algunas más frecuentes que otras al igual que pasa con la distribución de las categorías